Resultados esperados:
 - input: 
    - avg_runtime
    - input_complexity
    - speedup
    - eficiencia
 - output: 
    - p_cores


In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [3]:
df_clean = pd.read_csv("dataset/clean_data.csv")
df_clean['cores'] = df_clean['cores']/df_clean['cores'].max()
df_clean['input'] = df_clean['input']/df_clean['input'].max()

In [4]:
X = df_clean[['input', 'average_runtime', 'speedup', 'efficiency']].values
y = df_clean['cores'].values

### Regression Models

#### MLP Regression Model

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

In [11]:
learning_rate = 0.001
input_shape = (X_train.shape[1],)

callback = EarlyStopping(monitor="loss", patience=10)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(16, activation='relu', input_shape = input_shape))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X_train, y_train, batch_size=32, epochs=500, verbose=2, validation_data=(X_test, y_test), callbacks=[callback])

Epoch 1/500
7/7 - 0s - loss: 0.0938 - val_loss: 0.0661 - 350ms/epoch - 50ms/step
Epoch 2/500
7/7 - 0s - loss: 0.0789 - val_loss: 0.0540 - 20ms/epoch - 3ms/step
Epoch 3/500
7/7 - 0s - loss: 0.0697 - val_loss: 0.0477 - 20ms/epoch - 3ms/step
Epoch 4/500
7/7 - 0s - loss: 0.0628 - val_loss: 0.0472 - 26ms/epoch - 4ms/step
Epoch 5/500
7/7 - 0s - loss: 0.0584 - val_loss: 0.0440 - 23ms/epoch - 3ms/step
Epoch 6/500
7/7 - 0s - loss: 0.0552 - val_loss: 0.0422 - 20ms/epoch - 3ms/step
Epoch 7/500
7/7 - 0s - loss: 0.0526 - val_loss: 0.0398 - 21ms/epoch - 3ms/step
Epoch 8/500
7/7 - 0s - loss: 0.0506 - val_loss: 0.0383 - 20ms/epoch - 3ms/step
Epoch 9/500
7/7 - 0s - loss: 0.0491 - val_loss: 0.0380 - 21ms/epoch - 3ms/step
Epoch 10/500
7/7 - 0s - loss: 0.0476 - val_loss: 0.0367 - 22ms/epoch - 3ms/step
Epoch 11/500
7/7 - 0s - loss: 0.0473 - val_loss: 0.0360 - 22ms/epoch - 3ms/step
Epoch 12/500
7/7 - 0s - loss: 0.0453 - val_loss: 0.0360 - 21ms/epoch - 3ms/step
Epoch 13/500
7/7 - 0s - loss: 0.0444 - val_loss

In [5]:
loss = model.evaluate(X_test, y_test)
print(f'Erro no conjunto de teste: {loss}')

predictions = model.predict(X_test)

for i in range(10):
    print(f'Valor Real: {y_test[i]}, Previsão: {predictions[i][0]}')

3/3 [==============================] - 0s 3ms/step - loss: 0.0042
Erro no conjunto de teste: 0.00423419289290905
3/3 [==============================] - 0s 3ms/step
Valor Real: 0.1875, Previsão: 0.14959311485290527
Valor Real: 0.90625, Previsão: 0.8766377568244934
Valor Real: 0.59375, Previsão: 0.6690213680267334
Valor Real: 0.71875, Previsão: 0.6568863987922668
Valor Real: 0.84375, Previsão: 0.8327068090438843
Valor Real: 0.1875, Previsão: 0.1388242393732071
Valor Real: 0.0625, Previsão: 0.04049396887421608
Valor Real: 0.03125, Previsão: 0.008417685516178608
Valor Real: 0.5625, Previsão: 0.550567090511322
Valor Real: 0.96875, Previsão: 0.9392473697662354


#### MLP Regression Model with Leave-One-Out-Validation (LOOCV)

In [5]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error

loo = LeaveOneOut()
mse_scores = []
best_model = None
best_mse = float('inf')
callback = EarlyStopping(monitor="loss", patience=3)

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    learning_rate = 0.001
    input_shape = (X_train.shape[1],)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.Dense(8, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, y_train, epochs=10, verbose=0, callbacks=[callback])

    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)
    mse_scores.append(mse)

    if mse < best_mse:
        best_model = model
        best_mse = mse
    
mean_mse = np.mean(mse_scores)

print(f'Erro médio quadrático usando LOOCV: {mean_mse}')

1/1 [==============================] - 0s 29ms/step
Erro médio quadrático usando LOOCV: 0.05590887520104189


In [6]:
min(mse_scores)

6.272115946615031e-07